# Example Seldon Core Deployments using Helm
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
 - [seldon-core Python package](https://pypi.org/project/seldon-core/) (```pip install seldon-core>=0.2.6.1```)
 - [Helm client](https://helm.sh/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

## Setup

In [1]:
!kubectl create namespace seldon

namespace/seldon created


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


In [3]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


## Install Helm

In [4]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount/tiller created
clusterrolebinding.rbac.authorization.k8s.io/tiller created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [5]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


## Start seldon-core

In [6]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

NAME:   seldon-core-crd
LAST DEPLOYED: Wed Mar 13 08:30:20 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        1        1           0          0s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/Pod(related)
NAME                                         READY  STATUS             RESTARTS  AGE
seldon-spartakus-volunteer-5554c4d8b6-kmv5g  0/1    ContainerCreating  0         0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     4s


NOTES:
NOTES: TODO



In [7]:
!helm install ../helm-charts/seldon-core --name seldon-core --namespace seldon  --set ambassador.enabled=true

NAME:   seldon-core
LAST DEPLOYED: Wed Mar 13 08:30:28 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/ServiceAccount
NAME    SECRETS  AGE
seldon  1        0s

==> v1/Role
NAME          AGE
seldon-local  0s
ambassador    0s

==> v1/RoleBinding
NAME        AGE
ambassador  0s
seldon      0s

==> v1/Service
NAME                          TYPE       CLUSTER-IP     EXTERNAL-IP  PORT(S)                        AGE
seldon-core-ambassador-admin  NodePort   10.111.130.72  <none>       8877:31541/TCP                 0s
seldon-core-ambassador        NodePort   10.107.52.77   <none>       80:32072/TCP,443:30133/TCP     0s
seldon-core-seldon-apiserver  NodePort   10.111.115.91  <none>       8080:30459/TCP,5000:32436/TCP  0s
seldon-core-redis             ClusterIP  10.104.36.225  <none>       6379/TCP                       0s

==> v1beta1/Deployment
NAME                                DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-core-ambassador              1        1        1       

In [8]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador

Waiting for deployment "seldon-core-seldon-cluster-manager" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-seldon-cluster-manager" successfully rolled out
deployment "seldon-core-seldon-apiserver" successfully rolled out
Waiting for deployment "seldon-core-ambassador" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


## Set up REST and gRPC methods

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

## Serve Single Model

In [9]:
!helm install ../helm-charts/seldon-single-model --name mymodel

NAME:   mymodel
LAST DEPLOYED: Wed Mar 13 08:36:42 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME     AGE
mymodel  0s




In [10]:
!helm template ../helm-charts/seldon-single-model | pygmentize -l json

---
# Source: seldon-single-model/templates/model.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "RELEASE-NAME"
    },
    "spec": {
        "name": "RELEASE-NAME",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                                }
                            }
                        ],
                        "terminationGracePe

In [13]:
!kubectl rollout status deploy/mymodel-mymodel-7cd068f

Waiting for deployment "mymodel-mymodel-7cd068f" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymodel-mymodel-7cd068f" successfully rolled out


### Get predictions

In [1]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mymodel",namespace="seldon")

#### REST Request

In [2]:
r = sc.predict(gateway="ambassador",transport="rest")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.43531007521220577
  }
}

Response:
meta {
  puid: "thskm7ap6ah4gkknh02j7lhgg0"
  requestPath {
    key: "mymodel"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.07717553097868499
  }
}



#### gRPC Request

In [16]:
r = sc.predict(gateway="ambassador",transport="grpc")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.514969932920143
  }
}

Response:
meta {
  puid: "u52deu7590e9dc0or8e6l4m3p8"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08304340353650001
  }
}



In [17]:
!helm delete mymodel --purge

release "mymodel" deleted


## Serve AB Test

In [18]:
!helm install ../helm-charts/seldon-abtest --name myabtest

NAME:   myabtest
LAST DEPLOYED: Wed Mar 13 08:40:48 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME      AGE
myabtest  0s




In [19]:
!helm template ../helm-charts/seldon-abtest | pygmentize -l json

---
# Source: seldon-abtest/templates/ab_test_1pod.json


---
# Source: seldon-abtest/templates/ab_test_2pods.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
	"labels": {
	    "app": "seldon"
	},
	"name": "RELEASE-NAME"
    },
    "spec": {
	"name": "RELEASE-NAME",
	"predictors": [
	    {
		"name": "abtest",
		"replicas": 1,
		"componentSpecs": [{
		    "spec": {
			"containers": [
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-1",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }],
			"terminationGracePeriodSeconds": 20
		    }},
	        {
		    "metadata":{
			"labels":{
			    "version":"v2"
			}
		    },    
			"spec":{
			    "containers":[
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-2",
				"resources": {
				    "requests": {
					"memo

In [20]:
!kubectl rollout status deploy/myabtest-abtest-41de5b8
!kubectl rollout status deploy/myabtest-abtest-df66c5c

Waiting for deployment "myabtest-abtest-41de5b8" rollout to finish: 0 of 1 updated replicas are available...
deployment "myabtest-abtest-41de5b8" successfully rolled out
deployment "myabtest-abtest-df66c5c" successfully rolled out


### Get predictions

In [21]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="myabtest",namespace="seldon")

#### REST Request

In [22]:
r = sc.predict(gateway="ambassador",transport="rest")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.8531704639234817
  }
}

Response:
meta {
  puid: "gicn4qaks4mc55mkjm55cruo3d"
  routing {
    key: "myabtest"
    value: 1
  }
  requestPath {
    key: "classifier-2"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "myabtest"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.11269575212992548
  }
}



#### gRPC Request

In [23]:
r = sc.predict(gateway="ambassador",transport="grpc")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.13775105372602903
  }
}

Response:
meta {
  puid: "ap78beg3imign359hnpamdc1nn"
  routing {
    key: "myabtest"
  }
  requestPath {
    key: "classifier-1"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "myabtest"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.058474227887821084
  }
}



In [24]:
!helm delete myabtest --purge

release "myabtest" deleted


## Serve Multi-Armed Bandit

In [25]:
!helm install ../helm-charts/seldon-mab --name mymab

NAME:   mymab
LAST DEPLOYED: Wed Mar 13 08:45:00 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME   AGE
mymab  0s




In [26]:
!helm template ../helm-charts/seldon-mab | pygmentize -l json

---
# Source: seldon-mab/templates/mab.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
	"labels": {
	    "app": "seldon"
	},
	"name": "RELEASE-NAME"
    },
    "spec": {
	"name": "RELEASE-NAME",
	"predictors": [
	    {
		"name": "abtest",
		"replicas": 1,
		"componentSpecs": [{
		    "spec": {
			"containers": [
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-1",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }],
			"terminationGracePeriodSeconds": 20
		    }},
	        {
		    "metadata":{
			"labels":{
			    "version":"v2"
			}
		    },    
			"spec":{
			    "containers":[
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-2",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }
			],
			"terminationGracePeriodSeco

In [27]:
!kubectl rollout status deploy/mymab-abtest-41de5b8
!kubectl rollout status deploy/mymab-abtest-b8038b2
!kubectl rollout status deploy/mymab-abtest-df66c5c 

Waiting for deployment "mymab-abtest-41de5b8" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymab-abtest-41de5b8" successfully rolled out
deployment "mymab-abtest-b8038b2" successfully rolled out
deployment "mymab-abtest-df66c5c" successfully rolled out


### Get predictions

In [28]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mymab",namespace="seldon")

#### REST Request

In [29]:
r = sc.predict(gateway="ambassador",transport="rest")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.0004345728312186159
  }
}

Response:
meta {
  puid: "2bv45ejso0qpckooklhku894i5"
  routing {
    key: "eg-router"
  }
  requestPath {
    key: "classifier-1"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "eg-router"
    value: "seldonio/mab_epsilon_greedy:1.1"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.05135696112577633
  }
}



#### gRPC Request

In [31]:
r = sc.predict(gateway="ambassador",transport="grpc")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.9933641893212712
  }
}

Response:
meta {
  puid: "ts2hvors1433fk88fmgb47bm3j"
  routing {
    key: "eg-router"
  }
  requestPath {
    key: "classifier-1"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "eg-router"
    value: "seldonio/mab_epsilon_greedy:1.1"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.12749374958176052
  }
}



In [32]:
!helm delete mymab --purge

release "mymab" deleted
